## Timeseries of globally-integrated energy flux from barotropic to baroclinic tides

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

print("numpy version  : ", np.__version__)
print("xarray version : ", xr.__version__)

numpy version  :  1.26.4
xarray version :  2024.5.0


In [2]:
import os, sys
sys.path.append('../../')

In [3]:
from CM4X_metadata import *
exp_dict

{'CM4p25': {'hgrid': '/archive/Raphael.Dussin/datasets/OM4p25/c192_OM4_025_grid_No_mg_drag_v20160808_unpacked/ocean_hgrid.nc',
  'piControl-spinup': 'odiv-210',
  'piControl': 'odiv-230',
  'piControl-continued': 'odiv-306',
  'historical': 'odiv-231',
  'ssp585': 'odiv-232'},
 'CM4p125': {'hgrid': '/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc',
  'piControl-spinup': 'odiv-209',
  'piControl': 'odiv-313',
  'piControl-continued': 'odiv-437',
  'historical': 'odiv-255',
  'ssp585': 'odiv-293'}}

In [4]:
import doralite
import gfdl_utils.core as gu

In [5]:
def load_dataset(exp_name):
    pp = doralite.dora_metadata(exp_name)['pathPP']
    ppname = gu.find_unique_variable(pp, "TKE_itidal")
    out = "ts"
    local = gu.get_local(pp, ppname, out)
    static = gu.open_static(pp, ppname)
    ds = gu.open_frompp(pp, ppname, out, local, time="*", add="TKE_itidal", dmget=True)
    offset_years = 1850-101 if "piControl" in pp else 0
    ds = ds.assign_coords({'year': ds.time.dt.year + offset_years})
    return xr.merge([ds, static])

In [6]:
cutoff_depth=2500

ds_dict = {}
for model, exps in exp_dict.items():
    ds_dict[model] = {}
    for exp, exp_name in exps.items():
        if exp=='hgrid': continue
        
        data_path = f"../../data/plotting_ready/TKE_itidal_global_{model}_{exp}.nc"
        ds = load_dataset(exp_name)
        ds['TKE_itidal_areaweighted'] = ds.TKE_itidal*ds.areacello
        ds['TKE_itidal_global'] = ds.TKE_itidal_areaweighted.sum(['xh', 'yh']).compute()*1e-12

        ncells = 8 if model=="CM4p25" else 16
        ds['deptho_smooth'] = ds.deptho.fillna(0.).rolling({"xh":ncells, "yh":ncells}).mean()

        ds['TKE_itidal_deep'] = (
            ds.TKE_itidal_areaweighted
            .where(ds.deptho_smooth > cutoff_depth)
        ).sum(["xh", "yh"]).compute()*1e-12
        ds['TKE_itidal_shallow'] = ds.TKE_itidal_global - ds.TKE_itidal_deep
        ds.drop_dims(['nv', 'xh', 'yh', 'xq', 'yq']).to_netcdf(data_path, mode="w")
            
        ds_dict[model][exp] = ds

Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
Issuing dmget command to migrate data to disk. Migration complete.
